Read CSV data with Pandas.

In [4]:
import pandas as pd

# team stats
team_summaries = pd.read_csv('nba_data/stats_archive/Team Summaries.csv')
# team_totals = pd.read_csv('nba_data/stats_archive/Team Totals.csv')
team_per100 = pd.read_csv('nba_data/stats_archive/Team Stats Per 100 Poss.csv')

# player stats
# player_totals = pd.read_csv('nba_data/stats_archive/Player Totals.csv')
player_shooting = pd.read_csv('nba_data/stats_archive/Player Shooting.csv')
player_play_by_play = pd.read_csv('nba_data/stats_archive/Player Play By Play.csv')
player_per100 = pd.read_csv('nba_data/stats_archive/Per 100 Poss.csv')

# games
games = pd.read_csv('nba_data/game_archive/games.csv')
teamids = pd.read_csv('nba_data/game_archive/teams.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'nba_data/stats_archive/Team Summaries.csv'

Remove unnecessary features.

In [124]:
# games
teamids['TEAM_NAME'] = teamids['CITY'] + ' ' + teamids['NICKNAME']
teamids = teamids[['TEAM_ID','TEAM_NAME','ABBREVIATION']]

games = games[['GAME_DATE_EST','GAME_ID','HOME_TEAM_ID','VISITOR_TEAM_ID','SEASON','HOME_TEAM_WINS','PTS_home','PTS_away']]

In [125]:
# team stats
team_stats = team_summaries.merge(team_per100)
team_stats.drop(columns=['playoffs','arena','attend','lg'], inplace=True)

# player stats
player_stats = player_shooting.merge(player_play_by_play).merge(player_per100)
player_stats.drop(columns=['birth_year','pos','lg'], inplace=True)

In [127]:
team_stats

,season,team,abbreviation,age,w,l,pw,pl,mov,sos,...,ft_percent,orb_per_100_poss,drb_per_100_poss,trb_per_100_poss,ast_per_100_poss,stl_per_100_poss,blk_per_100_poss,tov_per_100_poss,pf_per_100_poss,pts_per_100_poss
0,2024,Atlanta Hawks,ATL,26.3,34.0,40.0,34.0,40.0,-1.36,-0.17,...,0.802,12.7,31.7,44.3,26.2,7.5,4.6,13.1,18.3,118.2
1,2024,Boston Celtics,BOS,28.3,58.0,16.0,59.0,15.0,11.38,-0.33,...,0.809,10.9,36.5,47.4,27.2,6.8,6.6,12.3,16.7,123.3
2,2024,Brooklyn Nets,BRK,26.3,29.0,46.0,32.0,43.0,-2.37,-0.08,...,0.757,11.7,33.1,44.8,26.7,7.2,5.1,13.2,19.2,113.7
3,2024,Chicago Bulls,CHI,28.0,36.0,39.0,34.0,41.0,-1.48,-0.25,...,0.794,11.2,33.4,44.6,25.7,7.8,5.0,12.5,19.2,114.7
4,2024,Charlotte Hornets,CHO,25.2,18.0,56.0,16.0,58.0,-10.55,-0.16,...,0.779,9.9,31.7,41.6,25.2,6.9,4.5,14.1,18.6,109.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397,1974,San Antonio Spurs,SAA,24.6,45.0,39.0,45.0,39.0,0.87,-0.13,...,0.778,15.3,32.8,48.2,22.4,7.4,2.8,16.5,27.3,101.7
1398,1974,San Diego Conquistadors,SDA,25.1,37.0,47.0,36.0,48.0,-2.54,0.26,...,0.753,15.5,29.6,45.2,21.5,7.5,5.6,14.3,17.8,102.9
1399,1974,Seattle SuperSonics,SEA,26.0,36.0,46.0,34.0,48.0,-2.55,0.26,...,0.767,14.6,29.8,44.4,23.2,7.6,3.2,17.9,22.9,96.8
1400,1974,Utah Stars,UTS,27.8,51.0,33.0,43.0,41.0,0.36,-0.08,...,0.789,13.8,31.6,45.4,22.4,8.4,4.2,17.1,23.2,105.6


Create player efficiency indices.

𝐼NBA=𝑁pts+𝑁rbs+𝑁asts+𝑁stls+𝑁blcks−(𝑁msFld+𝑁msFT+𝑁tos)

In [ ]:
player_stats['eff_ind'] = (player_stats['pts_per_100_poss'] +
                           player_stats['trb_per_100_poss'] +
                           player_stats['ast_per_100_poss'] +
                           player_stats['stl_per_100_poss'] +
                           player_stats['blk_per_100_poss'] -
                           (player_stats['tov_per_100_poss'] +
                            player_stats['fga_per_100_poss'] - player_stats['fg_per_100_poss'] +
                            player_stats['fta_per_100_poss'] - player_stats['ft_per_100_poss']))

player_stats

Create team efficiency indices.

In [ ]:
#create the team efficeny index by summing the players by season and team 
team_efficiency_index = player_stats.groupby(['tm', 'season'])['eff_ind'].sum().reset_index(name='team_efficiency_index')
#merging the team efficeny index to team_stats
team_stats = team_stats.merge(team_efficiency_index, left_on=['abbreviation', 'season'], right_on=['tm', 'season'], how='left')
team_stats.drop(columns = ["tm"], inplace=True)
team_stats = team_stats.dropna()
team_stats

In [ ]:
final_df = team_stats[(team_stats['season'] >= 2018) & (team_stats['season'] <= 2021)]
final_df

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = final_df.drop(['team', 'abbreviation', 'season', 'w'], axis=1)  # Features
y = final_df['w']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.naive_bayes import GaussianNB 
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Initialize models for regression
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(),
    "KNN Regressor": KNeighborsRegressor(),
    "SVR": SVR(),
    "XGBoost Regressor": XGBRegressor(objective='reg:squarederror'),
    "Neural Network Regressor": MLPRegressor(max_iter=1000)
}

results = {}

for name, model in models.items():
    pipeline = make_pipeline(StandardScaler(), model)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    results[name] = mse
    print(f"{name} MSE: {mse:.4f}")

# Displaying all results
print("\nModel Mean Squared Errors:")
for name, mse in results.items():
    print(f"{name}: {mse:.4f}")
